# Three output products for one eq
* 8-bit and 16-bit of same image S.T. psnr is high
* pairs of grayscale for super resolution (2 ways)
    * higher resolution by pixel size
    * higher resolution by pixel depth & pixel size
* edge mapping for both
    
## Generate and display:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import time
import sys
import tempfile

import skimage.io as im_io

from PIL import TiffImagePlugin as tip
from PIL.TiffImagePlugin import Image

import matplotlib.pyplot as pyplot

sys.path.insert(0, '../src/')
sys.path.insert(0, 'scalygraphic/src/')

from im_scale_products import *
from impute_color import *

def get_test_domain_dict(it_max=None, scale_dist=None):
    """ Usage: 
    domain_dict = get_center_domain_dict(it_max, scale_dist)
    """
    if it_max is None:
        it_max = 64
        
    if scale_dist is None:
        scale_dist = 12
        
    domain_dict = get_default_domain_dict()
    domain_dict['it_max'] = it_max
    domain_dict['max_d'] = scale_dist / domain_dict['zoom']
    domain_dict['n_rows'] = 255
    domain_dict['n_cols'] = 255
    
    return domain_dict

print('\t\t\tDomain dictionary for test:')
domain_dict = get_test_domain_dict()
for k, v in domain_dict.items():
    print(k,v)
    
print('\n\t\t\tEquations enumeration of (Name, function_handle) tuples')    
for k, v in name_functionhandle_dict.items():
    print(k, v)

In [ ]:
for fcn_number in range(len(name_functionhandle_dict)):
    print(name_functionhandle_dict[fcn_number][0], type(name_functionhandle_dict[fcn_number][1]))

In [ ]:
domain_dict = get_test_domain_dict()

for fcn_number in range(len(name_functionhandle_dict)):
    t0 = time.time()
    fcn_name, fcn = name_functionhandle_dict[fcn_number][0], name_functionhandle_dict[fcn_number][1]
    p = p = fcn(0.0, None)
    
    list_tuple = [(fcn, (p))]
    
    ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
    Zd_n2, Zr_n2, ETn_n2 = etg_norm(Z0, Z, ET)

    g_im_et = primitive_2_gray(ETn_n2)
    # complex result vectors: distance component
    g_im_Zd = primitive_2_gray(Zd_n2)

    # complex result vectors: rotation component
    g_im_Zr = primitive_2_gray(Zr_n2)

    im_gray = get_gray_im(ET, Z, Z0)
    
    im_list = [g_im_et, g_im_Zd, g_im_Zr, im_gray]
    new_im = cat_im_list_hori(im_list)
    tt = time.time() - t0
    
    print('\n')
    print('%20s: %0.6f seconds\n\t\tp ='%(fcn_name, tt), p)
    print('%15s%30s%30s%30s'%('ET','Zd','Zr', 'All'))
    display(new_im)
